# A Neural Algorithm of Artistic Style

这文章是图片风格迁移的第一篇文章，文章大部分内容都在讲废话，凑字数，真正有价值的内容其实并不多。**核心思想就是将 Content 图片 和 Style 图片过一遍 Convolutional Neural Network，取其中特定几层的输出，然后用定义出的 Content Loss 和 Style Loss，在新的图片上进行优化**.这个方法可以实现图像风格的迁移，但是实际的体验下来实在是太慢了，在 GPU 的环境下，300 个 Epoche 也需要 28s 渲染一张图片。这个 notebook 是结合 Pytorch 的官方例子对原始论文的一种解读。

## 环境准备

根据机器上的 CUDA 版本，安装对应 Pytroch 就好，我的机器是 Ubuntu 18.04， Cuda 10.2 所以根据[官方链接](https://pytorch.org/get-started/locally/)，我选择安装 10.1 版本的 Pytorch：

```
    conda install pytorch torchvision cudatoolkit=10.1 -c pytorch
```

Convolution Nerual Network 的每一层的输出单元，可以看作是抽取图片上某一部分特征的功能单元。如果解决的问题是图像识别，Neural Network 每一层做的事情就是在前一层的基础上对原始数据进行表示，这样随着处理层次的加深，相应层次就变的更关注图片的内容及内容在图片上所处的为位置而不是图片上具体的每一个像素的值。相应的，在 Neural Network 的较低层级，这些层级就是简单的从原始图片中提取像素值。

## Methods
